We can expand $\hat{H}_{dse}$ in terms of the dipole operator (with electronic and nuclear contributions) and dipole expectation values as follows:
\begin{align}
    \hat{H}_{dse} &= \frac{1}{2} \sum_{\xi, \xi'} \sum_{i \neq j} \lambda^{\xi} \lambda^{\xi'} \mu^{\xi}(x_i) \mu^{\xi'}(x_j) \\
    &+  \frac{1}{2} \sum_{\xi, \xi'} \sum_i \frac{1}{2} \lambda^{\xi} \lambda^{\xi'} Q^{\xi \xi'}(x_i) \\ 
   & + \left(\lambda \cdot \mu_{nuc} - \lambda \cdot \langle \mu \rangle \right) \sum_{\xi} \sum_i \lambda^{\xi} \mu^{\xi} (x_i)  \\
  &+ \frac{1}{2} \left( \lambda \cdot \mu_{nuc} \right)^2  - \left( \lambda \cdot \langle \mu \rangle \right) \left( \lambda \cdot \mu_{nuc} \right) + \frac{1}{2} \left( \lambda \cdot \langle \mu \rangle \right)^2
\end{align} 

In the above expansion of $\hat{H}_{dse}$ we have specifically indicated that the product of electronic dipole operators contains 2-electron contributions when $i \neq j$,
and 1-electron quadrupole contributions when $i = j$.  Furthermore, a
one-electron term arises that contains the electronic dipole operator
scaled by $\lambda \cdot \mu_{nuc}$.  In the QED-RHF procedure, the additional one-electron terms above will be added to $H_{core}$ and the additional two-electron terms above will be included in the density-matrix dependent terms in the Fock operator:
\begin{equation}
    F_{\mu \nu} = H_{\mu \nu} + G_{\mu \nu}
\end{equation}
where
\begin{align}
    H_{\mu \nu} &= h_{\mu \nu} - \frac{1}{2}\sum_{\xi, \xi'} \lambda^{\xi} \lambda^{\xi'} Q^{\xi \xi'}_{\mu \nu} \\
    &+ \left(\lambda \cdot \mu_{nuc} - \lambda \cdot \langle \mu \rangle \right) \sum_{xi} \lambda^{\xi} \mu^{\xi}_{\mu \nu}
\end{align}
and
\begin{align}
  G_{\mu \nu} &=   
  \left( 2(\mu\,\nu\left|\,\lambda\,\sigma) - (\mu\,\lambda\,\right|\nu\,\sigma) \right) D_{\lambda\sigma} \\
  & + \left( \sum_{\xi \xi'} \lambda^{\xi} \lambda^{\xi'} \left(\mu^{\xi}_{\mu \nu} \mu^{\xi'}_{\lambda \sigma} - \frac{1}{2} \mu^{\xi}_{\mu \lambda} \mu^{\xi'}_{\nu \sigma} \right)
  \right)D_{\lambda\sigma},
\end{align}
leading to the total QED-RHF energy being
\begin{equation}
    E_{QED-RHF} =  (F_{\mu\nu} + H_{\mu\nu})D_{\mu\nu} + E_{nuc} + d_c
\end{equation}
where $d_c =\frac{1}{2} \left( \lambda \cdot \mu_{nuc} \right)^2  - \left( \lambda \cdot \langle \mu \rangle \right) \left( \lambda \cdot \mu_{nuc} \right) + \frac{1}{2} \left( \lambda \cdot \langle \mu \rangle \right)^2$
Note: In the above, the quadrupole terms have a negative sign!  This is because we are really using the quadrupole integrals in place of products of dipole integrals, but these differ by a factor of the electron charge which is negative 1 in atomic units.  For example:
\begin{equation}
\hat{Q}_{el}^{xx} \propto q_{el} \hat{x} \hat{x}
\end{equation}
whereas 

\begin{equation}
\hat{\mu}_{el}^x \hat{\mu}_{el}^x \propto q_{el} q_{el} \hat{x} \hat{x}
\end{equation}

Hence, we use the quadrupole integrals but multiply by -1 to compensate!


This notebook uses the CQED-RHF function written within `helper_cqed_rhf.py` and runs
it on a case for which the result is known from Table I and II results [here](https://arxiv.org/pdf/2011.12768.pdf).
There is an assert statement that tests that the energy between our code and the known answer agrees to within 5e-5 Hartrees, which is the error due to the fact that a density fitting approximation was used in the Eugene's paper which we do not employ here!

In [5]:
from __future__ import print_function

"""
A reference implementation of cavity quantum electrodynamics 
configuration interactions singles.
"""

__authors__   = ["Jon McTague", "Jonathan Foley"]
__credits__   = ["Jon McTague", "Jonathan Foley"]

__copyright_amp__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2021-01-15"

# ==> Import Psi4, NumPy, & SciPy <==
import psi4
import numpy as np
import scipy.linalg as la
import time
from helper_cqed_rhf import *

# Set Psi4 & NumPy Memory Options
psi4.set_memory('2 GB')
#psi4.core.set_output_file('output.dat', False)

numpy_memory = 2

# basis set etc
psi4.set_options({'basis':        'def2-tzvppd',
                  'scf_type':     'pk',
                  'reference':    'rhf',
                  'mp2_type':     'conv',
                  'save_jk': True,
                  'e_convergence': 1e-10,
                  'd_convergence': 1e-10})

# MgH+ string
mol_string = """

0 1
    NA           0.000000000000     0.000000000000    -0.875819904077
    F            0.000000000000     0.000000000000     1.059820520433
no_reorient
#nocom
symmetry c1
"""

psi4.geometry(mol_string)
eugenes_answer =  -261.371070718358
# electric field
Ex = 0.
Ey = 0.
Ez = 0.01


#Ez = (lam / np.sqrt(2.0 * frequency))
lam = np.array([Ex, Ey, Ez])

psi4_rhf_energy, wfn = psi4.energy('scf', return_wfn=True)

# run cqed_rhf and capture the results!
rhf_e, cqed_rhf_e, cqed_vecs = cqed_rhf(lam, mol_string, self_consistent_dipole=True)
print("Original RHF Energy:     ", rhf_e)
print("Eugene's CQED-RHF Energy:", eugenes_answer)
print("Our CQED-RHF Energy:     ", cqed_rhf_e)
assert np.isclose(cqed_rhf_e, eugenes_answer,5e-5)



-261.37134961184677

Start SCF iterations:

Canonical RHF One-electron energy = -423.8943127302466110
CQED-RHF One-electron energy = -423.8915963779146523
Nuclear repulsion energy = 27.0652251280564968
Dipole energy = 0.0004869813724925
SCF Iteration   1: Energy = -261.3711084073713664   dE = -2.61371E+02   dRMS = 2.23432E-06
SCF Iteration   2: Energy = -261.3711084455142100   dE = -3.81428E-08   dRMS = 1.52094E-06
SCF Iteration   3: Energy = -261.3711084501623532   dE = -4.64814E-09   dRMS = 9.16648E-07
SCF Iteration   4: Energy = -261.3711084512467551   dE = -1.08440E-09   dRMS = 7.11218E-07
SCF Iteration   5: Energy = -261.3711084516505707   dE = -4.03816E-10   dRMS = 5.18558E-07
SCF Iteration   6: Energy = -261.3711084518432131   dE = -1.92642E-10   dRMS = 3.90748E-07
SCF Iteration   7: Energy = -261.3711084519444512   dE = -1.01238E-10   dRMS = 2.91707E-07
SCF Iteration   8: Energy = -261.3711084519998735   dE = -5.54223E-11   dRMS = 2.18703E-07
SCF Iteration   9: Energy = -261.37